<a href="https://colab.research.google.com/github/naidu970/Automated-LinkedIn-Job-Listing-Scraping-with-Google-Sheets-Integration/blob/main/webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated LinkedIn Job Listing Scraping with Google Sheets Integration




In [ ]:
!pip install gspread gspread_dataframe  -q

#importing required packages and libraries
import csv
import requests
from bs4 import BeautifulSoup
import json
import tqdm
import pandas as pd
from datetime import datetime
import gspread
from gspread_dataframe import set_with_dataframe
import numpy as np


In [ ]:

def convert_date(date):
    """
    Convert a date string to a formatted date string

    Parameters:
        date (str): A date string in ISO format.
    Returns:
        str: A formatted date and time string in the format "YYYY-MM-DD HH:MM:SS".

    """
    date_string = date[:-1]  #Removing Z
    parsed_date = datetime.fromisoformat(date_string)
    formatted_date = parsed_date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date

def process_text(text, replace_tupl):
    """
    Replace substrings in the given text based on a tuple of tuples.

    Parameters:
        text (str): The input text to be processed.
        replace_tupl (tuple of tuple): A tuple of tuples, where each tuple contains two elements:
            - The substring to be replaced.
            - The replacement substring.

    Returns:
        str: The text with replacements applied based on the provided tuple of tuples.
    """
    for pair in replace_tupl:
        text = text.replace(pair[0],pair[1])
    return text


In [ ]:
# url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/jobs-in-worldwide?start="
url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/jobs-in-worldwide?keywords=&f_TPR=&f_JT=F%2CC&f_WT=2&start="

url_list=[] # Collect Listing Urls
id_list = [] #Collect Listing Ids
num = 25 #Number of Linkeldn posts per scroll
start_num = 0 #starting number
end_num = start_num + 1000
for i in tqdm.tqdm(range(start_num, end_num, num)):
    try:
        response = requests.get(url+str(i))  #Sending request
    except:
        continue
    soup = BeautifulSoup(response.content,'html.parser')

    #finding the required data from the tags
    job_title_elements = soup.find_all('a', class_='base-card__full-link')
    job_id_elements= soup.find_all('div', attrs={"data-entity-urn": True})
    job_location = soup.find_all('span',class_='job-search-card__location')


    for job_loc,job_id,job_title in zip(job_location,job_id_elements,job_title_elements):
        if ("United States" in job_loc.text) or  ("Canada" in job_loc.text): #No job selection from United States and Canada
            continue
        #appending extracted data to the lists
        id_list.append(job_id['data-entity-urn'].split(":")[-1])
        url_list.append(job_title.get('href'))


print("Url :",url_list[0])
print("Listing ID :",id_list[0])

100%|███████████████████████████████████████████| 40/40 [01:15<00:00,  1.88s/it]

Url : https://br.linkedin.com/jobs/view/assistente-administrativo-at-epharma-pbm-phygital-3746989994?refId=FOqnlTqA1AIs%2FeCYGBsUeQ%3D%3D&trackingId=DTYHRBBf%2B5BLdtDepz%2BKrw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
Listing ID : 3746989994


In [ ]:
#DataFrame to store extracted data
df = pd.DataFrame({'company_id':[],
                'listing_id':[],
             'listing_url':[],
             'job_posting_title':[],
             'seniority_level':[],
             'employment_type':[],
             'job_function':[],
             'industry':[],
             'date_posted':[],
             'description':[],
             'hiring_organization_name':[],
             'hiring_organization_url':[],
             'job_location_address_type':[],
             'job_location_address_country':[],
             'job_location_address_locality':[],
             'job_name':[],
             'job_location_address_locality':[],
              'job_post_valid_date':[]
             })

#tuple of tuple to process text
replace_tupl = (('li',''),('li',''),(';strong',''),('/strong',''),('ul',''),('gt',' '),('br',''),\
                ('lt',''),('/',''),('&',''),(';',''),('  ','\n'),('\n\n','\n'))



for num in tqdm.tqdm(range(len(url_list))):

    try:
        job_post_link = url_list[num]
        job_id = id_list[num]

        response = requests.get(job_post_link)  #Sending request
        soup = BeautifulSoup(response.content,'html.parser')

        #finding the required data from the tags
        job_attr_val= soup.find_all('span', class_='description__job-criteria-text description__job-criteria-text--criteria')
        job_attr_name = soup.find_all('h3', class_='description__job-criteria-subheader')
        job_attr = {attr_name : 'not applicable' for attr_name in ['Seniority level','Employment type','Job function','Industries']}

        for name, val in (zip(job_attr_name,job_attr_val)):
            #extracting and storing data
            job_attr[name.get_text(strip=True)] = val.get_text(strip=True)


        try:
            #extract job post essential data as string
            str_json = soup.find('script',{'type':'application/ld+json'}).text
            #loading into json file
            dict_job = json.loads(str_json)

            #Job Location except from US and CA
            if (dict_job['jobLocation']['address']['addressCountry'] in ["US","CA"]):
                continue
            #appending to the dataframe
            df.loc[len(df)] = (soup.find("meta",{'name':'companyId'})['content'],\
                               job_id,\
                               f"https://www.linkedin.com/jobs/view/{job_id}",\
                               soup.find('title').text,\
                               job_attr['Seniority level'],\
                               job_attr['Employment type'],\
                               job_attr['Job function'],\
                               job_attr['Industries'],\
                               convert_date(dict_job['datePosted']),\
                               process_text(dict_job['description'],replace_tupl),\
                               dict_job['hiringOrganization']['name'],\
                               dict_job['hiringOrganization']['sameAs'],\
                               dict_job['jobLocation']['address']['@type'],\
                               dict_job['jobLocation']['address']['addressCountry'],\
                               dict_job['jobLocation']['address']['addressLocality'],\
                               dict_job['title'],\
                               convert_date(dict_job['validThrough']))
        except AttributeError:
            pass

    except :
        print("bad request")

 20%|████████                                 | 173/874 [02:56<10:06,  1.16it/s]

In [ ]:
#shape of Dataframe
print(df.shape)

In [ ]:
#using api to authorize changes in file
sa = gspread.service_account(filename="python-web-scraping-403619-fcfb4d21578b.json")
#selecting google sheets file
sheet =sa.open("linkeldn-job-listing")
#selecting required sheet
worksheet = sheet.worksheet("Sheet1")

#columns to find duplicates on
columns = ['company_id','listing_id', 'hiring_organization_name', 'job_name', 'listing_url',\
       'job_location_address_country', 'job_location_address_locality']

In [ ]:
try:
    #converting sheet into dataframe
    df_existing = pd.DataFrame(worksheet.get_all_records())
    #concating existing dataframe and newly acquired data
    df_tmp =pd.concat([df_existing,df],axis=0).reset_index(drop=True)
    #removing duplicates
    sub_df = df_tmp[~df_tmp.duplicated(columns, keep='first')].reset_index(drop=True)
    #uploading new data + old data into the sheet
    set_with_dataframe(worksheet=worksheet, dataframe=sub_df, include_index=False,\
                       include_column_header=True, resize=True)
except IndexError:
    #as sheet is empty initally
    print('Index Error')
    #uploading new data into sheet
    set_with_dataframe(worksheet=worksheet, dataframe=df, include_index=False,\
                       include_column_header=True, resize=True)